## Planet: Understanding the Amazon from Space ·

Use satellite data to track the human footprint in the Amazon rainforest.

#### Kaggle 
- https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/data


In [2]:
import numpy as np 
import pandas as pd 
import os
import cv2
from tqdm import tqdm

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input, merge, UpSampling2D, Cropping2D, ZeroPadding2D, Reshape, core, Convolution2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import optimizers
from keras import backend as K
from keras.optimizers import SGD
from keras.layers.merge import concatenate

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
x_train = []
x_test = []
y_train = []

path = ""
name = "Unet"
weights_path = path + name + '.h5'

In [7]:
# load data
df_train = pd.read_csv('/Users/chrisjcc/Desktop/DataInsight/data/planet_understanding_the_amazon_from_space/train_v2.csv')
df_test = pd.read_csv('/Users/chrisjcc/Desktop/DataInsight/data/planet_understanding_the_amazon_from_space/sample_submission_v2.csv')


In [8]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

In [9]:
labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}


In [10]:
img_size = 64
channels = 4 #4 for tiff, 3 for jpeg

In [17]:
for f, tags in tqdm(df_test.values, miniters=1000):
    print(f)
    img = cv2.imread('/Users/chrisjcc/Desktop/DataInsight/data/planet_understanding_the_amazon_from_space/test-tif-v2/{}.tif'.format(f), -1)
    x_test.append(cv2.resize(img, (img_size, img_size)))
x_test  = np.array(x_test, np.float32)/255.

  0%|          | 0/61191 [00:00<?, ?it/s]

test_0
test_1
test_2
test_3
test_4
test_5
test_6
test_7
test_8
test_9
test_10
test_11
test_12
test_13
test_14
test_15
test_16
test_17
test_18
test_19
test_20
test_21
test_22
test_23
test_24
test_25
test_26
test_27
test_28
test_29
test_30
test_31
test_32
test_33
test_34
test_35
test_36
test_37
test_38
test_39
test_40
test_41
test_42
test_43
test_44
test_45
test_46
test_47
test_48
test_49
test_50
test_51
test_52
test_53
test_54
test_55
test_56
test_57
test_58
test_59
test_60
test_61
test_62
test_63
test_64
test_65
test_66
test_67
test_68
test_69
test_70
test_71
test_72
test_73
test_74
test_75
test_76
test_77
test_78
test_79
test_80
test_81
test_82
test_83
test_84
test_85
test_86
test_87
test_88
test_89
test_90
test_91
test_92
test_93
test_94
test_95
test_96
test_97
test_98
test_99
test_100
test_101
test_102
test_103
test_104
test_105
test_106
test_107
test_108
test_109
test_110
test_111
test_112
test_113
test_114
test_115
test_116
test_117
test_118
test_119
test_120
test_121
test_122
tes

  2%|▏         | 1000/61191 [00:03<03:21, 298.65it/s]

test_977
test_978
test_979
test_980
test_981
test_982
test_983
test_984
test_985
test_986
test_987
test_988
test_989
test_990
test_991
test_992
test_993
test_994
test_995
test_996
test_997
test_998
test_999
test_1000
test_1001
test_1002
test_1003
test_1004
test_1005
test_1006
test_1007
test_1008
test_1009
test_1010
test_1011
test_1012
test_1013
test_1014
test_1015
test_1016
test_1017
test_1018
test_1019
test_1020
test_1021
test_1022
test_1023
test_1024
test_1025
test_1026
test_1027
test_1028
test_1029
test_1030
test_1031
test_1032
test_1033
test_1034
test_1035
test_1036
test_1037
test_1038
test_1039
test_1040
test_1041
test_1042
test_1043
test_1044
test_1045
test_1046
test_1047
test_1048
test_1049
test_1050
test_1051
test_1052
test_1053
test_1054
test_1055
test_1056
test_1057
test_1058
test_1059
test_1060
test_1061
test_1062
test_1063
test_1064
test_1065
test_1066
test_1067
test_1068
test_1069
test_1070
test_1071
test_1072
test_1073
test_1074
test_1075
test_1076
test_1077
test_1078
tes

  3%|▎         | 2000/61191 [00:07<03:27, 285.85it/s]

...

file_8899
file_89
file_890
file_8900
file_8901
file_8902
file_8903
file_8904
file_8905
file_8906
file_8907
file_8908
file_8909
file_891
file_8910
file_8911
file_8912
file_8913
file_8914
file_8915
file_8916
file_8917
file_8918
file_8919
file_892
file_8920
file_8921
file_8922
file_8923
file_8924
file_8925
file_8926
file_8927
file_8928
file_8929
file_893
file_8930
file_8931
file_8932
file_8933
file_8934
file_8935
file_8936
file_8937
file_8938
file_8939
file_894
file_8940
file_8941
file_8942
file_8943
file_8944
file_8945
file_8946
file_8947
file_8948
file_8949
file_895
file_8950
file_8951
file_8952
file_8953
file_8954
file_8955
file_8956
file_8957
file_8958
file_8959
file_896
file_8960
file_8961
file_8962
file_8963
file_8964
file_8965
file_8966
file_8967
file_8968
file_8969
file_897
file_8970
file_8971
file_8972
file_8973
file_8974
file_8975
file_8976
file_8977
file_8978
file_8979
file_898
file_8980
file_8981
file_8982
file_8983
file_8984
file_8985
file_8986
file_8987
file_8988
file_8989
f

100%|█████████▉| 61000/61191 [04:10<00:00, 254.34it/s]

file_9782
file_9783
file_9784
file_9785
file_9786
file_9787
file_9788
file_9789
file_979
file_9790
file_9791
file_9792
file_9793
file_9794
file_9795
file_9796
file_9797
file_9798
file_9799
file_98
file_980
file_9800
file_9801
file_9802
file_9803
file_9804
file_9805
file_9806
file_9807
file_9808
file_9809
file_981
file_9810
file_9811
file_9812
file_9813
file_9814
file_9815
file_9816
file_9817
file_9818
file_9819
file_982
file_9820
file_9821
file_9822
file_9823
file_9824
file_9825
file_9826
file_9827
file_9828
file_9829
file_983
file_9830
file_9831
file_9832
file_9833
file_9834
file_9835
file_9836
file_9837
file_9838
file_9839
file_984
file_9840
file_9841
file_9842
file_9843
file_9844
file_9845
file_9846
file_9847
file_9848
file_9849
file_985
file_9850
file_9851
file_9852
file_9853
file_9854
file_9855
file_9856
file_9857
file_9858
file_9859
file_986
file_9860
file_9861
file_9862
file_9863
file_9864
file_9865
file_9866
file_9867
file_9868
file_9869
file_987
file_9870
file_9871
file_9872
f

100%|██████████| 61191/61191 [04:11<00:00, 243.63it/s]

file_9977
file_9978
file_9979
file_998
file_9980
file_9981
file_9982
file_9983
file_9984
file_9985
file_9986
file_9987
file_9988
file_9989
file_999
file_9990
file_9991
file_9992
file_9993
file_9994
file_9995
file_9996
file_9997
file_9998
file_9999


In [18]:
for f, tags in tqdm(df_train.values, miniters=1000):
    #https://stackoverflow.com/questions/37512119/resize-transparent-image-in-opencv-python-cv2
    #If you load a 4 channel image, the flag -1 indicates that the image is loaded unchanged, so you can load and split all 4 channels directly.
    img = cv2.imread('/Users/chrisjcc/Desktop/DataInsight/data/planet_understanding_the_amazon_from_space/train-tif-v2/{}.tif'.format(f), -1)#0-1 voir au dessus les 2 comments
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (img_size, img_size)))
    y_train.append(targets)

100%|██████████| 40479/40479 [02:43<00:00, 247.50it/s]


In [19]:
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32)/255.

print(x_train.shape)
print(y_train.shape)

(40479, 64, 64, 4)
(40479, 17)


In [20]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2)   

print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_val), len(Y_val))

Split train:  32383 32383
Split valid:  8096 8096


In [21]:
def get_crop_shape(target, refer):
        # width, the 3rd dimension
        cw = (target.get_shape()[2] - refer.get_shape()[2]).value
        assert (cw >= 0)
        if cw % 2 != 0:
            cw1, cw2 = int(cw/2), int(cw/2) + 1
        else:
            cw1, cw2 = int(cw/2), int(cw/2)
        # height, the 2nd dimension
        ch = (target.get_shape()[1] - refer.get_shape()[1]).value
        assert (ch >= 0)
        if ch % 2 != 0:
            ch1, ch2 = int(ch/2), int(ch/2) + 1
        else:
            ch1, ch2 = int(ch/2), int(ch/2)

        return (ch1, ch2), (cw1, cw2)

In [22]:
def get_unet(n_ch,patch_height,patch_width):
    concat_axis = 3

    inputs = Input((patch_height, patch_width, n_ch))
    
    conv1 = Conv2D(32, (3, 3), padding="same", name="conv1_1", activation="relu", data_format="channels_last")(inputs)
    conv1 = Conv2D(32, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)
    conv2 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool1)
    conv2 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)

    conv3 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool2)
    conv3 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv3)

    conv4 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool3)
    conv4 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv4)

    conv5 = Conv2D(512, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool4)
    conv5 = Conv2D(512, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv5)

    up_conv5 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv5)
    ch, cw = get_crop_shape(conv4, up_conv5)
    crop_conv4 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv4)
    up6   = concatenate([up_conv5, crop_conv4], axis=concat_axis)
    conv6 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(up6)
    conv6 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv6)

    up_conv6 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv6)
    ch, cw = get_crop_shape(conv3, up_conv6)
    crop_conv3 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv3)
    up7   = concatenate([up_conv6, crop_conv3], axis=concat_axis)
    conv7 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(up7)
    conv7 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv7)

    up_conv7 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv7)
    ch, cw = get_crop_shape(conv2, up_conv7)
    crop_conv2 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv2)
    up8   = concatenate([up_conv7, crop_conv2], axis=concat_axis)
    conv8 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(up8)
    conv8 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv8)

    up_conv8 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv8)
    ch, cw = get_crop_shape(conv1, up_conv8)
    crop_conv1 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv1)
    up9   = concatenate([up_conv8, crop_conv1], axis=concat_axis)
    conv9 = Conv2D(32, (3, 3), padding="same", activation="relu", data_format="channels_last")(up9)
    conv9 = Conv2D(32, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv9)

    #ch, cw = get_crop_shape(inputs, conv9)
    #conv9  = ZeroPadding2D(padding=(ch[0],cw[0]), data_format="channels_last")(conv9)
    #conv10 = Conv2D(1, (1, 1), data_format="channels_last", activation="sigmoid")(conv9)
    
    flatten =  Flatten()(conv9)
    Dense1 = Dense(512, activation='relu')(flatten)
    BN =BatchNormalization() (Dense1)
    Dense2 = Dense(17, activation='sigmoid')(BN)
    
    model = Model(input=inputs, output=Dense2)
    
    return model

In [23]:
model = get_unet(channels, img_size, img_size)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [24]:
epochs_arr  = [   20,      5,      5]
learn_rates = [0.001, 0.0003, 0.0001]

In [ ]:
for learn_rate, epochs in zip(learn_rates, epochs_arr):
    if os.path.isfile(weights_path):
        print("loading existing weight for training")
        model.load_weights(weights_path)
    
    opt  = optimizers.Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                  optimizer=opt,
                  metrics=['accuracy'])
    callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=1),
                 ModelCheckpoint(weights_path, monitor='val_loss', save_best_only=True, verbose=2)]

    model.fit(x = X_train, y= Y_train, validation_data=(X_val, Y_val),
          batch_size=256, verbose=2, epochs=epochs, callbacks=callbacks, shuffle=True)

if os.path.isfile(weights_path):
    model.load_weights(weights_path)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 32383 samples, validate on 8096 samples
Epoch 1/20
 - 1777s - loss: 0.3606 - accuracy: 0.8798 - val_loss: 0.4710 - val_accuracy: 0.9084

Epoch 00001: val_loss improved from inf to 0.47105, saving model to Unet.h5
Epoch 2/20
 - 25545s - loss: 0.1681 - accuracy: 0.9341 - val_loss: 0.1820 - val_accuracy: 0.9277

Epoch 00002: val_loss improved from 0.47105 to 0.18204, saving model to Unet.h5
Epoch 3/20


In [ ]:
p_val = model.predict(X_val, batch_size = 128, verbose=1)
print(fbeta_score(Y_val, np.array(p_val) > 0.2, beta=2, average='samples'))

p_test = model.predict(x_test, batch_size = 128, verbose=1)

In [ ]:
result = p_test
result = pd.DataFrame(result, columns = labels)

Additional info: http://deeplearning.net/tutorial/fcn_2D_segm.html
